In [2]:
import torch
import torch.nn as nn

In [3]:
from tqdm import tqdm

In [8]:
from torchvision.datasets import FashionMNIST

In [13]:
dataset_train = FashionMNIST(root="../datasets/", train=True, download=False)
dataset_test = FashionMNIST(root="../datasets/", train=False, download=False)

In [17]:
train_X = dataset_train.data.type(torch.float)
train_Y = dataset_train.targets

test_X = dataset_test.data.type(torch.float)
test_Y = dataset_test.targets

In [19]:
train_X.shape

torch.Size([60000, 28, 28])

In [51]:
class alexnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=97, kernel_size=(11, 11), stride=(4, 4)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)),
            nn.Conv2d(in_channels=97, out_channels=256, kernel_size=(5,5), padding=(2,2)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)),
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=(3,3), padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=(3,3),padding=(1,1)),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3,3), padding=(1,1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3,3), stride=(2,2)), 
            nn.Flatten(),
            nn.Linear(in_features=256*5*5, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=1000),
            nn.LogSoftmax(dim=1)
        )
    def forward(self, x):
        return self.layer_stack(x)

In [52]:
x = torch.concat([torch.randn(224,224)]*3).reshape(1, 3, 224, 224)
x.shape
model = alexnet()
model(x).shape

torch.Size([1, 1000])